# Detección de patrones de fraude en transacciones

Una empresa fintech quiere analizar transacciones bancarias para detectar posibles fraudes.
Cada transacción tiene:

* Monto

* Ubicación

* Hora

* Categoría del gasto

* Descripción opcional

Tu tarea es usar embeddings vectoriales para detectar transacciones sospechosas, comparando una transacción nueva con patrones históricos.

## Objetivo

1. Convertir transacciones históricas en vectores (combinar atributos numéricos y textuales).

2. Guardarlas en una base vectorial.

3. Evaluar nuevas transacciones comparando su vector con los históricos.

4. Indicar cuáles son potencialmente sospechosas, basándose en la similitud con fraudes previos.

In [1]:
#1. Convertir transacciones históricas en vectores (combinar atributos numéricos y textuales)

# Importar librerías necesarias
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb

# Cargar el dataset de transacciones
df = pd.read_csv('Data/transacciones.csv')
print(df.head(10))
print(f"\nTotal de transacciones: {len(df)}")
print(f"Fraudes detectados: {df['es_fraude'].sum()}")
print(f"Transacciones legítimas: {(df['es_fraude'] == 0).sum()}")

c:\Users\ST09\Documents\GitHub\STEMIA\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


   id  monto ubicacion   hora        categoria  \
0   1   5000      Lima  23:45     Electrónicos   
1   2    200      Lima  10:30           Comida   
2   3   4500     Cusco  00:15     Electrónicos   
3   4    150  Arequipa  13:20       Transporte   
4   5   6000      Lima  22:50     Electrónicos   
5   6     50     Cusco  08:15           Comida   
6   7   7000      Lima  23:10     Electrónicos   
7   8    300  Arequipa  09:45       Transporte   
8   9   1200      Lima  20:30  Entretenimiento   
9  10   4800     Cusco  00:05     Electrónicos   

                       descripcion  es_fraude  
0      Compra online internacional          1  
1            Desayuno en cafetería          0  
2                  Compra nocturna          1  
3                      Taxi urbano          0  
4         Pedido online alto valor          1  
5                       Café y pan          0  
6   Orden de gadgets internacional          1  
7                             Uber          0  
8             Ent

In [2]:
#2. Crear embeddings combinando información numérica y textual

# Inicializar el modelo de embeddings para texto
model = SentenceTransformer('all-MiniLM-L6-v2')

# Estrategia: Combinar datos numéricos y textuales en un texto descriptivo
# Esto permite capturar patrones como: "compra nocturna + electrónicos + monto alto"
def crear_texto_transaccion(row):
    """Convierte una transacción en texto descriptivo para embedding"""
    return f"""Transacción de {row['monto']} soles 
    en {row['ubicacion']} 
    a las {row['hora']} 
    categoría {row['categoria']} 
    {row['descripcion']}"""

# Generar textos descriptivos
df['texto_completo'] = df.apply(crear_texto_transaccion, axis=1)

# Crear embeddings del texto completo
embeddings = model.encode(df['texto_completo'].tolist())
df['embedding'] = embeddings.tolist()

print(f"Embeddings generados: {len(embeddings)} vectores de dimensión {embeddings.shape[1]}")
print(f"\nEjemplo de texto generado:\n{df['texto_completo'].iloc[0]}")

Embeddings generados: 20 vectores de dimensión 384

Ejemplo de texto generado:
Transacción de 5000 soles 
    en Lima 
    a las 23:45 
    categoría Electrónicos 
    Compra online internacional


In [3]:
#3. Almacenar en base de datos vectorial ChromaDB

# Crear cliente y colección
client = chromadb.Client()
collection = client.create_collection(
    name="transacciones_fraude",
    metadata={"description": "Transacciones históricas para detección de fraude"}
)

# Insertar todas las transacciones con sus metadatos
for idx, row in df.iterrows():
    collection.add(
        documents=[row['texto_completo']],
        embeddings=[row['embedding']],
        metadatas=[{
            'id': int(row['id']),
            'monto': float(row['monto']),
            'ubicacion': row['ubicacion'],
            'hora': row['hora'],
            'categoria': row['categoria'],
            'descripcion': row['descripcion'],
            'es_fraude': int(row['es_fraude'])
        }],
        ids=[str(row['id'])]
    )

In [4]:
#4. Evaluar nuevas transacciones y detectar fraudes

def analizar_transaccion(monto, ubicacion, hora, categoria, descripcion, n_similares=5):
    """
    Analiza una nueva transacción comparándola con el histórico
    
    Parámetros:
    - monto: cantidad de la transacción
    - ubicacion: ciudad/lugar
    - hora: hora de la transacción (formato HH:MM)
    - categoria: tipo de gasto
    - descripcion: descripción adicional
    - n_similares: número de transacciones similares a buscar
    
    Retorna:
    - nivel_riesgo: "ALTO", "MEDIO", "BAJO"
    - score_fraude: porcentaje de fraudes en transacciones similares
    - transacciones_similares: lista de casos parecidos
    """
    
    # 1. Crear texto descriptivo de la nueva transacción
    texto_nueva = f"""Transacción de {monto} soles 
    en {ubicacion} 
    a las {hora} 
    categoría {categoria} 
    {descripcion}"""
    
    # 2. Generar embedding
    embedding_nueva = model.encode([texto_nueva])
    
    # 3. Buscar transacciones similares en el histórico
    results = collection.query(
        query_embeddings=embedding_nueva.tolist(),
        n_results=n_similares
    )
    
    # 4. Analizar los resultados
    transacciones_similares = []
    fraudes_encontrados = 0
    
    print(f"\n{'='*80}")
    print(f"ANÁLISIS DE NUEVA TRANSACCIÓN")
    print(f"{'='*80}")
    print(f"Monto: ${monto}")
    print(f"Ubicación: {ubicacion}")
    print(f"Hora: {hora}")
    print(f"Categoría: {categoria}")
    print(f"Descripción: {descripcion}")
    print(f"\n{'─'*80}")
    print(f"TRANSACCIONES SIMILARES EN EL HISTÓRICO:")
    print(f"{'─'*80}\n")
    
    for i in range(len(results['ids'][0])):
        metadata = results['metadatas'][0][i]
        es_fraude = metadata['es_fraude']
        fraudes_encontrados += es_fraude
        
        transacciones_similares.append({
            'id': metadata['id'],
            'monto': metadata['monto'],
            'ubicacion': metadata['ubicacion'],
            'hora': metadata['hora'],
            'categoria': metadata['categoria'],
            'es_fraude': es_fraude
        })
        
        # Mostrar resultado
        icono = "FRAUDE" if es_fraude else "LEGÍTIMA"
        print(f"{i+1}. {icono}")
        print(f"   ID: {metadata['id']} | Monto: ${metadata['monto']} | {metadata['ubicacion']}")
        print(f"   Hora: {metadata['hora']} | Cat: {metadata['categoria']}")
        print(f"   Desc: {metadata['descripcion']}")
        print()
    
    # 5. Calcular score de riesgo
    score_fraude = (fraudes_encontrados / n_similares) * 100
    
    # 6. Determinar nivel de riesgo
    if score_fraude >= 60:
        nivel_riesgo = "ALTO"
        recomendacion = "BLOQUEAR y revisar manualmente"
    elif score_fraude >= 40:
        nivel_riesgo = "MEDIO"
        recomendacion = "Solicitar verificación adicional"
    else:
        nivel_riesgo = "BAJO"
        recomendacion = "Aprobar transacción"
    
    # 7. Mostrar conclusión
    print(f"{'='*80}")
    print(f"RESULTADO DEL ANÁLISIS")
    print(f"{'='*80}")
    print(f"Nivel de Riesgo: {nivel_riesgo}")
    print(f"Score de Fraude: {score_fraude:.1f}%")
    print(f"Fraudes similares encontrados: {fraudes_encontrados}/{n_similares}")
    print(f"Recomendación: {recomendacion}")
    print(f"{'='*80}\n")
    
    return nivel_riesgo, score_fraude, transacciones_similares


# EJEMPLO 1: Transacción sospechosa (patrón de fraude)
print("=" * 80)
print("CASO 1: Transacción potencialmente fraudulenta")
print("=" * 80)
analizar_transaccion(
    monto=5500,
    ubicacion="Lima",
    hora="23:30",
    categoria="Electrónicos",
    descripcion="Compra online nocturna de productos caros"
)

# EJEMPLO 2: Transacción normal
print("\n" + "=" * 80)
print("CASO 2: Transacción legítima")
print("=" * 80)
analizar_transaccion(
    monto=180,
    ubicacion="Lima",
    hora="12:00",
    categoria="Comida",
    descripcion="Almuerzo en restaurante"
)

CASO 1: Transacción potencialmente fraudulenta

ANÁLISIS DE NUEVA TRANSACCIÓN
Monto: $5500
Ubicación: Lima
Hora: 23:30
Categoría: Electrónicos
Descripción: Compra online nocturna de productos caros

────────────────────────────────────────────────────────────────────────────────
TRANSACCIONES SIMILARES EN EL HISTÓRICO:
────────────────────────────────────────────────────────────────────────────────

1. FRAUDE
   ID: 12 | Monto: $5500.0 | Arequipa
   Hora: 22:30 | Cat: Electrónicos
   Desc: Pedido grande en tienda online

2. FRAUDE
   ID: 5 | Monto: $6000.0 | Lima
   Hora: 22:50 | Cat: Electrónicos
   Desc: Pedido online alto valor

3. FRAUDE
   ID: 3 | Monto: $4500.0 | Cusco
   Hora: 00:15 | Cat: Electrónicos
   Desc: Compra nocturna

4. FRAUDE
   ID: 1 | Monto: $5000.0 | Lima
   Hora: 23:45 | Cat: Electrónicos
   Desc: Compra online internacional

5. FRAUDE
   ID: 7 | Monto: $7000.0 | Lima
   Hora: 23:10 | Cat: Electrónicos
   Desc: Orden de gadgets internacional

RESULTADO DEL ANÁLIS

('BAJO',
 0.0,
 [{'id': 11,
   'monto': 100.0,
   'ubicacion': 'Lima',
   'hora': '11:00',
   'categoria': 'Comida',
   'es_fraude': 0},
  {'id': 2,
   'monto': 200.0,
   'ubicacion': 'Lima',
   'hora': '10:30',
   'categoria': 'Comida',
   'es_fraude': 0},
  {'id': 9,
   'monto': 1200.0,
   'ubicacion': 'Lima',
   'hora': '20:30',
   'categoria': 'Entretenimiento',
   'es_fraude': 0},
  {'id': 16,
   'monto': 250.0,
   'ubicacion': 'Lima',
   'hora': '12:30',
   'categoria': 'Comida',
   'es_fraude': 0},
  {'id': 6,
   'monto': 50.0,
   'ubicacion': 'Cusco',
   'hora': '08:15',
   'categoria': 'Comida',
   'es_fraude': 0}])

## Respuestas a las Preguntas

### 1. Vectorización:

**¿Cómo combinar atributos numéricos y textuales en un embedding?**

En este ejercicio usamos una estrategia de **texto enriquecido**: convertimos todos los atributos (numéricos y categóricos) en una descripción textual coherente. Por ejemplo:
```
"Transacción de 5000 soles en Lima a las 23:45 categoría Electrónicos Compra online internacional"
```

**Ventajas de esta aproximación:**
- ✅ Simple de implementar
- ✅ Captura relaciones semánticas naturales
- ✅ Los modelos de lenguaje entienden contexto numérico

**¿Qué pasa si solo usamos texto o solo números?**
- **Solo texto:** Perdemos información crítica como el monto (una compra de $5000 vs $50 es muy diferente)
- **Solo números:** Perdemos contexto semántico (no sabríamos que "Electrónicos" y "Gadgets" son similares)

**Alternativas más avanzadas:**
- Embeddings híbridos (concatenar embeddings textuales con features numéricos normalizados)
- Modelos multimodales entrenados específicamente para datos tabulares

---

### 2. Métrica de similitud:

**¿Conviene cosine similarity, Euclidean, o una combinación?**

En este caso usamos **Cosine Similarity** (por defecto en ChromaDB), y es la mejor opción porque:

✅ **Cosine Similarity:**
- Mide el ángulo entre vectores, no la magnitud
- Ideal para embeddings de texto (lo que nos importa es la dirección semántica)
- Robusto ante diferencias de escala
- Rango: -1 a 1 (más fácil de interpretar)

❌ **Distancia Euclidiana:**
- Sensible a la magnitud de los vectores
- Menos efectiva para embeddings de alta dimensión
- Puede dar más peso a features irrelevantes

**Recomendación:** Para embeddings de texto → **Cosine Similarity** siempre.

---

### 3. Interpretación:

**¿Cómo decidir si una transacción es sospechosa basándose en vecinos más cercanos?**

Implementamos un sistema de **scoring por mayoría votada**:

1. **Buscamos las 5 transacciones más similares** en el histórico
2. **Contamos cuántas son fraudes**
3. **Calculamos porcentaje:** `(fraudes_encontrados / total) * 100`

**Umbrales de decisión:**
- 🔴 **≥60% fraudes** → Riesgo ALTO → Bloquear
- 🟡 **40-59% fraudes** → Riesgo MEDIO → Verificación adicional  
- 🟢 **<40% fraudes** → Riesgo BAJO → Aprobar

**Lógica:** Si transacciones muy similares en el pasado fueron fraudes, esta probablemente también lo sea.

**Mejoras posibles:**
- Ponderar por distancia (dar más peso a vecinos más cercanos)
- Usar más vecinos (k=10 o k=20) para mayor robustez
- Aplicar machine learning supervisado con los embeddings como features

---

### 4. Opcional - Filtros y Visualización:

**Filtrar por ubicación o categoría antes de comparar:**

Podríamos usar el parámetro `where` de ChromaDB:
```python
results = collection.query(
    query_embeddings=embedding_nueva.tolist(),
    n_results=5,
    where={"categoria": "Electrónicos"}  # Solo buscar en electrónicos
)
```

**Ventajas:**
- ✅ Comparaciones más relevantes
- ✅ Reduce falsos positivos
- ✅ Más rápido en bases de datos grandes

**Desventajas:**
- ❌ Podría perder patrones cross-category
- ❌ Menos datos para comparar

**Visualización 2D:** Implementada con PCA en las celdas anteriores. Permite ver clusters de fraudes vs transacciones legítimas visualmente.